# Điều kiện: các mốc là cách đều nhau và số mốc sử dụng là một số lẻ

# 1. Code của chương trình. Khi sử dụng code thì "Run All" và thao tác ở task "Chạy chương trình"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def doc_input (ten_file):
    #tra ve gia tri cua x va y tu file input.txt
    # doc file input.txt
    inp = open(ten_file, "r")
    # doc du lieu cua x va y
    x = inp.readline()
    y = inp.readline()
    # xu ly du lieu cua x va y
    x = x.strip().split()
    x = np.array(x, dtype=float)
    if (y == ""):
        y = f(x)
        y = np.array(y, dtype=float)
        inp.close()
    else:
        y = y.strip().split()
        y = np.array(y, dtype=float)
        inp.close()
    return x, y
    
def hoocnerChia(a, value):
    b = []
    b.append(a[0])
    for i in range(1, len(a)):
        b.append(b[i - 1] * value + a[i])
    return b

def hoocnerNhanBosung(coeffPolyTich, t):
    if len(coeffPolyTich) == 0:
        coeffPolyTich.append(1)
        coeffPolyTich.append(-t)
    else:
        coeffPolyTich.append(0)
        temp = coeffPolyTich.copy()
        for i in range(1, len(coeffPolyTich)):
            coeffPolyTich[i] = coeffPolyTich[i] - t * temp[i - 1]
    return coeffPolyTich

def saiPhan(f_0, f_1, yk, x0, xk):
    if xk > x0:
        temp = f_0.copy()
        f_0.append(0)
        f_0[0] = yk
        for i in range(1, len(f_0)):
            f_0[i] = f_0[i - 1] - temp[i - 1]
        f_1.append(f_0[len(f_0) - 1])
    else:
        temp = f_1.copy()
        f_1.append(0)
        f_1[0] = yk
        for i in range(1, len(f_1)):
            f_1[i] = temp[i - 1] - f_1[i - 1]
        f_0.append(f_1[len(f_1) - 1])
    return f_0, f_1

def besselFormula(heSoDaThuc, coeffTich, f_0, f_1, k, x0, x1, x2, y1, y2, giaiThua_k):
    #da thuc theo an u
    if k == 0: #Buoc co so
        heSoDaThuc.append(y2 - y1)
        heSoDaThuc.append((y1 + y2)/2)
        f_0.append(y2)
        f_1.append(y1)
        f_0.append(y2 - y1)
        f_1.append(y2 - y1)
        giaiThua_k = 1
    else:
        f_0, f_1 = saiPhan(f_0, f_1, y1, x0, x1)
        f_0, f_1 = saiPhan(f_0, f_1, y2, x0, x2)

        #su dung dat p = u^2: p - 1/4; p - 9/4 ...; p - (k - 1)^2/4
        coeffTich = hoocnerNhanBosung(coeffTich, (k - 1)**2/4)

        giaiThua_k *= len(heSoDaThuc)
        c_chan = (f_1[len(f_1) - 2] + f_0[(len(f_0) - 2)]) / (2 * giaiThua_k)
        giaiThua_k *= len(heSoDaThuc) + 1
        c_le = f_1[len(f_1) - 1] / giaiThua_k
        
        #tinh he so da thuc k le
        heSoDaThucCu = heSoDaThuc.copy()
        heSoDaThuc.append(0)
        for j in range(1, len(heSoDaThuc)):
            if j % 2 == 0:
                heSoDaThuc[j] = heSoDaThucCu[j - 1] + c_chan * coeffTich[int(j / 2)]
            else:
                heSoDaThuc[j] = heSoDaThucCu[j - 1]
        heSoDaThuc[0] = c_chan

        #tinh he so da thuc k chan
        heSoDaThucCu = heSoDaThuc.copy()
        heSoDaThuc.append(0)
        for j in range(1, len(heSoDaThuc)):
            if j % 2 == 0:
                heSoDaThuc[j] = heSoDaThucCu[j - 1] + c_le * coeffTich[int((j + 1) / 2)]
            else:
                heSoDaThuc[j] = heSoDaThucCu[j - 1]
        heSoDaThuc[0] = c_le

    return heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k

In [4]:
def sortData(path):
    if path.endswith('.csv'):
        data = pd.read_csv(path)
        dataX = data['x']
        dataY = data['y']
    elif path.endswith('.txt'):
        print("su dung doc file .txt")
        dataX, dataY = doc_input("input.txt")
    for i in range(len(dataX)):
        indexMin = i
        for j in range(i + 1, len(dataX)):
            if dataX[j] < dataX[indexMin]:
                indexMin = j
        if indexMin != i:
            dataX[indexMin], dataX[i] = dataX[i], dataX[indexMin]
            dataY[indexMin], dataY[i] = dataY[i], dataY[indexMin]
    return dataX, dataY

def kiemTraMoc(dataX, x0):
    h = dataX[1] - dataX[0]
    m = round((x0 - dataX[0])/h)
    if x0 > dataX[len(dataX) - 1] or x0 < dataX[0] or x0 - (dataX[0] + m * h) > 1e-10:
        print("Moc noi suy duoc chon ban dau la khong hop le")
        print("Quay lai, chon lai moc noi suy")
    else:
        print("Moc noi suy ban dau hop le. Chuong trinh tiep tuc")

def checkSoMoc(dataX, soMoc, x0):
    h = dataX[1] - dataX[0]
    m = round((x0 - dataX[0])/h)
    if m <= len(dataX)/2:
        n = 2 * m + 1
    else:
        n = 2 * (len(dataX) - m) + 1
    if n > len(dataX):
        n = len(dataX)
        
    #check xem so moc chon co hop le hay khong
    if soMoc > n or soMoc <= 0 or soMoc % 2 == 1:
        print("so luong moc lua chon la khong hop le")
        print("Nhap lai so moc. So moc phai la so chan thuoc trong khoang tu:", 0, "->", n)
    else:
        print("so luong moc la hop le. Tiep tuc chuong trinh")
        
def sortGauss(dataX, dataY, x0, soMoc):
    x = []
    y = []
    m = round((x0 - dataX[0]) / (dataX[1] - dataX[0]))
    for i in range(soMoc):
        if i % 2 == 1:
            k = int((i + 1)/2)
            x.append(dataX[m + k])
            y.append(dataY[m + k])
        else:
            k = int(-(i + 1)/2)
            x.append(dataX[m + k])
            y.append(dataY[m + k])
    return x, y

In [5]:
def main(dataX, dataY, x0, soMoc):
    heSoDaThuc = []
    coeffTich = []
    f_0 = [] 
    f_1 = [] 
    giaiThua_k = 1
    h = dataX[1] - dataX[0]
    with open("output.txt", "w") as out:
        x, y = sortGauss(dataX, dataY, x0, soMoc)
        print("Các mốc đã sử dụng là: \nx = {0}\ny = {1}".format(x, y))
        for i in range(0, soMoc, 2):
            heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k = besselFormula(heSoDaThuc, coeffTich, f_0, f_1, i, x0, x[i], x[i + 1], y[i], y[i + 1], giaiThua_k)
            
            out.writelines("\n\n--------------------------------------------------------------------------------------")
            out.writelines("\nDa thuc moi sau khi them moc 2 noi suy: ({}, {}); ({}, {}): ".format(x[i], y[i], x[i + 1], y[i + 1]))
            out.write("\nTap cac moc noi suy: \t")
            for j in range(i + 2):
                out.write("({}, {}); \t".format(x[j], y[j]))
            out.write("\nDa thuc bac {}: \nP(t) =: ".format(i + 1))
            for j in range(len(heSoDaThuc)):
                out.write("{} * x^{} \t + \t".format(heSoDaThuc[j], len(heSoDaThuc) - 1 - j))
            out.write("\n\nThu lai\n")
            for j in range(i + 2):
                out.writelines("\nTai du lieu x = {}".format(x[j]))
                u = (x[j] - x[0])/h - 1/2
                a = hoocnerChia(heSoDaThuc, u)
                out.writelines("\nPn(x) - y = {}".format(a[len(heSoDaThuc) - 1] - y[j]))
    return heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k

# 2. Chạy chương trình

In [6]:
# Dữ liệu đầu vào
x, y = doc_input("input.txt")

# Mốc ban đầu:
x0 = 2.6

# Số lượng mốc sử dụng:
soMoc = 6

heSodaThuc, coeffTich, f_0, f_1, giaiThua_k =  main(x, y, x0, soMoc)
print("\nĐa thức nội suy có bậc là: ", len(heSodaThuc) - 1)
print("Hệ số của đa thức nội suy là: ")
for i in range(len(heSodaThuc)):
    print("{0} x^{1}".format(heSodaThuc[i], len(heSodaThuc)-1-i))

Các mốc đã sử dụng là: 
x = [2.6, 2.8, 2.4, 3.0, 2.2, 3.2]
y = [1.0938, 0.1662, 1.8885, 1.8179, 1.9034, 1.6924]

Đa thức nội suy có bậc là:  5
Hệ số của đa thức nội suy là: 
-0.07611666666666668 x^5
-0.10423750000000002 x^4
0.6423250000000001 x^3
0.8721937500000001 x^2
-1.0834239583333334 x^1
0.41846640625 x^0


## Tính giá trị đa thức nội suy tại một điểm bất kì

In [7]:
# Điểm cần tính giá trị => cần chọn mốc ban đầu x0 gần xk
xk = 3

h = x[1] - x[0]
u = (xk - x0)/h - 1/2
print("Giá trị của đa thức nội suy tại xk = {0}\nLà: {1}".format(xk, hoocnerChia(heSodaThuc, u).pop()))

Giá trị của đa thức nội suy tại xk = 3
Là: 1.817899999999995
